In [0]:
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/gdrive/My Drive/anime-faces.zip", 'r')
zip_ref.extractall("/content/data")
zip_ref.close()

In [0]:
import os
from skimage.transform import rescale
from skimage.transform import resize
list_file1 = os.listdir('data/data')
print(len(list_file1))
list_file = []
for i in list_file1:
    name_length = len(i)
    if(i[name_length-1]=="g"):
        list_file.append(i)
        
print(len(list_file))

21552
21551


In [0]:
import numpy as np
from skimage.io import imread
data_train_gan = np.array([resize(imread(os.path.join('data/data',file_name)), (64, 64)) for file_name in list_file])

In [0]:
learning_params = 0.0005
epochs = 50000
batch_size = 64

image_dim = 64*64*3
neural_network_dim = 4096
latent_variable_dim = 64

In [0]:
def xavier(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1./tf.sqrt(in_shape[0]/2.))
  return (val)

In [0]:
Weight = { "weight_matrix_encoder_hidden": tf.Variable(xavier([image_dim, neural_network_dim])),
           "weight_mean_hidden": tf.Variable(xavier([neural_network_dim, latent_variable_dim])),
           "weight_std_hidden": tf.Variable(xavier([neural_network_dim, latent_variable_dim])),
           "weight_matrix_decoder_hidden": tf.Variable(xavier([latent_variable_dim, neural_network_dim])),
           "weight_decoder": tf.Variable(xavier([neural_network_dim, image_dim]))
         }

Bias = { "bias_matrix_encoder_hidden": tf.Variable(xavier([neural_network_dim])),
         "bias_mean_hidden": tf.Variable(xavier([latent_variable_dim])),
         "bias_std_hidden": tf.Variable(xavier([latent_variable_dim])),
         "bias_matrix_decoder_hidden": tf.Variable(xavier([neural_network_dim])),
         "bias_decoder": tf.Variable(xavier([image_dim]))
       }

In [0]:
#Encoder section
image_x = tf.placeholder(tf.float32, shape=[None, image_dim])

Encoder_layer = tf.add(tf.matmul(image_x, Weight["weight_matrix_encoder_hidden"]), Bias["bias_matrix_encoder_hidden"])
Encoder_layer = tf.nn.tanh(Encoder_layer)

Mean_layer = tf.add(tf.matmul(Encoder_layer, Weight["weight_mean_hidden"]), Bias["bias_mean_hidden"])
Standard_deviation_layer = tf.add(tf.matmul(Encoder_layer, Weight["weight_std_hidden"]), Bias["bias_std_hidden"])

NameError: ignored

In [0]:
#Reparametrization Trick
epsilon = tf.random_normal(tf.shape(Standard_deviation_layer), dtype=tf.float32, mean=0.0, stddev = 1.0)
latent_layer = Mean_layer + tf.exp(0.5*Standard_deviation_layer)*epsilon

In [0]:
#Decoder Layer
Decoder_hidden = tf.add(tf.matmul(latent_layer, Weight["weight_matrix_decoder_hidden"]), Bias["bias_matrix_decoder_hidden"])
Decoder_hidden = tf.nn.tanh(Decoder_hidden)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden, Weight["weight_decoder"]), Bias["bias_decoder"])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

In [0]:
def loss_function(original_image, reconstructed_image):

  #Reconstruction loss
  data_fidelity_loss = original_image*tf.log(1e-10 + reconstructed_image) + (1-original_image)*tf.log(1e-10 + 1-reconstructed_image)
  data_fidelity_loss = -tf.reduce_sum(data_fidelity_loss,1)

  #KL Divergence Loss
  KL_div_loss = 1 + Standard_deviation_layer - tf.square(Mean_layer) - tf.exp(Standard_deviation_layer)
  KL_div_loss = -0.5*tf.reduce_sum(KL_div_loss,1)
  
  network_loss = tf.reduce_mean(data_fidelity_loss + KL_div_loss)
  return network_loss

In [0]:
loss_value = loss_function(image_x, Decoder_output_layer)
optimizer = tf.train.RMSPropOptimizer(learning_params).minimize(loss_value)

init = tf.global_variables_initializer()

In [0]:
sess.close()
sess = tf.Session()

In [0]:
sess.run(init)

In [0]:
start = 0

for i in range(epochs):
  stop = start + batch_size
  x_batch = data_train_gan[start: stop]
  for i in x_batch:
    i.flatten()
  _, loss = sess.run([optimizer, loss_value], feed_dict = {image_x : x_batch})
  start += batch_size
  if i%10 == 0:
    print("loss is {0} at iteration {1}".format(loss,i))

ValueError: ignored